1. 네이버 지도 음식점 검색
2. 필터 검색

In [2]:
import pandas as pd
import numpy as np
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains

In [33]:
# 검색어 입력 받아 검색하는 함수
def n_search(input_):
    # 크롬 드라이버 설정
    option = webdriver.ChromeOptions()
    option.add_argument('--start-maximized')
    driver = webdriver.Chrome('C:/Users/min/chromedriver.exe', options=option)
    action = ActionChains(driver)
    driver.get('https://map.naver.com/v5/')
    sleep(3)
    # 검색어 입력
    query = driver.find_element(By.CSS_SELECTOR,'#container > shrinkable-layout > div > app-base > search-input-box > div > div.search_box > div > input')
    query.send_keys(input_)
    query.send_keys(Keys.ENTER)
    sleep(5)
    # 음식점 정보
    restaurant_info = []
    # 메뉴 정보
    info_ = {}
    # 음식점 클릭을 위한 변수
    idx_1 = 1
    idx_3 = 0
    while True:
        try:
            # 새로운 프레임으로 변경
            driver.switch_to.default_content()
            element = driver.find_element(By.ID,"searchIframe")
            driver.switch_to.frame(element)
            sleep(2)
            # 음식점 클릭
            # 10 번째 음식점 이후로 태그 위치 변경됨
            if idx_3 == 0:
                driver.find_element(By.XPATH, f'//*[@id="_pcmap_list_scroll_container"]/ul/li[{idx_1}]/div[1]/a/div/div/span[1]').click()
                sleep(2)
                name = driver.find_element(By.XPATH, f'//*[@id="_pcmap_list_scroll_container"]/ul/li[{idx_1}]/div[1]/a/div/div/span[1]').text
            else:
                driver.find_element(By.XPATH, f'//*[@id="_pcmap_list_scroll_container"]/ul/div[{idx_3}]/li[{idx_1}]/div[1]/a/div/div/span[1]').click()
                sleep(2)
                name = driver.find_element(By.XPATH, f'//*[@id="_pcmap_list_scroll_container"]/ul/div[{idx_3}]/li[{idx_1}]/div[1]/a/div/div/span[1]').text
            # 새로운 프레임으로 변경
            driver.switch_to.default_content()
            element2 = driver.find_element(By.ID, "entryIframe")
            driver.switch_to.frame(element2)
            sleep(2)
            # 음식점 정보 추출
            try:
                star = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/em').text
            except:
                star = np.nan
            category = driver.find_element(By.XPATH, '//*[@id="_title"]/span[2]').text
            location = driver.find_element(By.XPATH, '//*[@class="IH7VW"]').text
            # 메뉴 클릭
            try:
                driver.find_element(By.XPATH, '//span[contains(text(), "메뉴")]').click()
            except:
                idx_1 += 1
                continue
            sleep(3)
            idx_2 = 1
            info = []
            plus_price = 0
            menu_count = 0
            # 메뉴 정보 추출
            while True:
                try:
                    menu = driver.find_element(By.XPATH, f'//*[@class="ZUYk_"]/li[{idx_2}]//*[@class="Sqg65"]').text
                    price = driver.find_element(By.XPATH, f'//*[@class="ZUYk_"]/li[{idx_2}]//*[@class="SSaNE"]').text
                    plus_price += int(price.replace('원', '').replace(',', ''))
                    menu_count += 1
                    info.append([menu, price])
                    idx_2 += 1
                except:
                    try:
                        driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[7]/div/div[1]/div[2]/a').click()
                        sleep(2)
                    except:
                        try:
                            avg_price = int(plus_price/menu_count)
                            restaurant_info.append([name, category, location, star, avg_price])
                        except:
                            pass
                        idx_1 +=1
                        break
                info_[name] = info
        except:
            idx_3 += 1
            # 더이상 음식점이 없을 때 break
            try:
                action.move_to_element(driver.find_element(By.XPATH, f'//*[@id="_pcmap_list_scroll_container"]/ul/div[{idx_3}]')).perform()
            except:
                break
            idx_1 = 1
            sleep(3)
    driver.quit()
    # 데이터 프레임화
    restaurant_table = pd.DataFrame(restaurant_info, columns=['가게이름', '카테고리', '위치', '별점', '평균가'])
    restaurant_table['별점'] = restaurant_table['별점'].astype(float)
    table = pd.DataFrame(columns=['가게이름', '메뉴', '가격'])
    for name in info_.keys():
        for i in range(0,len(info_[name])):
            table = table.append({'가게이름': name, '메뉴': info_[name][i][0], '가격': info_[name][i][1]},ignore_index=True,)
    # csv 파일로 저장
    restaurant_table.to_csv(f'C:\\Users\\min\\{input_}.csv', index=False)
    table.to_csv(f'C:\\Users\\min\\{input_}_restaurant.csv', index=False)
    return table, restaurant_table

In [32]:
# 필터 검색 함수
def r_filter(r, r_info, category='', star=0, price_low=0, price_high=99999999):
    """
    r = 음식점 메뉴 정보
    r_info = 음식점 정보
    category = 음식 종류
    star = 최소 별점
    price_low = 음식점 평균가 최소값
    price_low = 음식점 평균가 최대값
    """
    search_table = pd.DataFrame(columns=['가게이름', '메뉴', '가격'])
    for info in r_info.values:
        if category in info[1] and star <= info[3] and price_low <= info[4] and price_high >= info[4]:
            search_table = search_table.append(r[r['가게이름'] == info[0]])
    return search_table

In [4]:
# 안산 맛집 검색
ansan_r, ansan_r_info = n_search('안산맛집')

C:\Users\min\AppData\Local\Temp/ipykernel_13656/3024976343.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/min/chromedriver.exe', options=option)


In [25]:
# 안산 맛집 정보
ansan_r_info

,가게이름,카테고리,위치,별점,평균가
0,금화전복,전복요리,경기 안산시 단원구 다리간1길 16,4.34,37454
1,부드럽소,소고기구이,경기 안산시 단원구 광덕서로 78 대한법조,4.72,52333
2,금화전복,전복요리,경기 안산시 단원구 다리간1길 16,4.34,37454
3,조순금닭도리탕,닭요리,경기 안산시 상록구 본삼로 39 6층,4.38,26285
4,제주삼육돈 고잔점,돼지고기구이,경기 안산시 단원구 광덕2로 155-16 1층,4.59,31000
5,11호남도할머니칼국수,"칼국수,만두",경기 안산시 단원구 대선로 11 11호남도할머니칼국수,4.42,31142
6,카페루헤,"카페,디저트",경기 안산시 단원구 서위매봉길 32 카페루헤,4.48,7477
7,아리화,"육류,고기요리",경기 안산시 단원구 광덕서로 62 2층,4.28,15845
8,멘야하나비 안산점,일본식라면,경기 안산시 단원구 광덕대로 174 월드타운 B동 1층 111호,NaN,11222
9,롱텀,베이커리,경기 안산시 단원구 이삭2길 41 1층 롱텀베이커리,NaN,1500


In [24]:
# 안산 맛집 메뉴 정보
ansan_r

,가게이름,메뉴,가격
0,금화전복,전복삼합구이(중),"63,000원"
1,금화전복,전복갈비찜(중),"65,000원"
2,금화전복,전복삼합구이(대),"83,000원"
3,금화전복,전복갈비찜(대),"85,000원"
4,금화전복,황태구이정식(점심특선),"13,000원"
...,...,...,...
520,시골순대,술국,"11,000원"
521,시골순대,곱창만(내장),"11,000원"
522,시골순대,순대국밥,"10,000원"
523,시골순대,오소리머리국밥,"10,000원"


In [39]:
# 안산 맛집 중 양식, 별점 4점 이상, 평균가 10000원 이상인 곳 메뉴
r_filter(ansan_r, ansan_r_info, '양식', 4, 10000)

,가게이름,메뉴,가격
349,뽁식당 고잔점,이불덮은 밥,"6,000원"
350,뽁식당 고잔점,봉골레파스타,"14,000원"
351,뽁식당 고잔점,토봉이파스타,"15,000원"
352,뽁식당 고잔점,오감베로파스타,"14,000원"
353,뽁식당 고잔점,만조파스타,"15,000원"
354,뽁식당 고잔점,담백로제파스타,"14,000원"
355,뽁식당 고잔점,빠네파스타,"16,000원"
356,뽁식당 고잔점,쌈싸먹는피자,"20,000원"
357,뽁식당 고잔점,머쉬룸크림리조또,"14,000원"
358,뽁식당 고잔점,로제리조또,"14,000원"


In [22]:
# 강남 맛집 검색
gangnam_r, gangnam_r_info = n_search('강남맛집')

C:\Users\min\AppData\Local\Temp/ipykernel_13656/3026656498.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/min/chromedriver.exe', options=option)


In [26]:
# 강남 맛집 정보
gangnam_r_info

,가게이름,카테고리,위치,별점,평균가
0,고래불 역삼본점,생선회,서울 강남구 논현로79길 62 백악빌딩 1층 고래불,4.45,110000
1,강남진해장,한식,서울 강남구 테헤란로5길 11,4.34,33555
2,대우부대찌개 강남본점,"찌개,전골",서울 강남구 테헤란로25길 34,4.42,27000
3,런던 베이글 뮤지엄 도산점,베이커리,"서울 강남구 언주로168길 33 1, 2층",NaN,5260
4,꽁티드툴레아,브런치,서울 강남구 도산대로49길 39,NaN,12357
5,땀땀,베트남음식,서울 강남구 강남대로98길 12-5,4.39,12500
6,중앙실비,"곱창,막창,양",서울 강남구 학동로 342 SK허브 1층,NaN,20500
7,이끼롤까스 가로수길점,돈가스,서울 강남구 강남대로156길 35,4.52,13200
8,노티드 청담,카페,서울 강남구 도산대로53길 15 1층,4.44,3500
9,소스윗예스,퓨전음식,서울 강남구 강남대로162길 27-16 1층,NaN,12631


In [23]:
# 강남 맛집 메뉴 정보
gangnam_r

,가게이름,메뉴,가격
0,고래불 역삼본점,바다바람,"125,000원"
1,고래불 역삼본점,바다풍경,"95,000원"
2,강남진해장,양선지해장국,"11,000원"
3,강남진해장,곱창전골,"62,000원"
4,강남진해장,양지내장탕,"14,000원"
...,...,...,...
561,달마시안,달마시안 한줄떡볶이,"19,000원"
562,달마시안,피쉬앤칩스,"22,000원"
563,달마시안,드립커피,"6,000원"
564,달마시안,생 초콜릿 라떼,"8,000원"


In [29]:
# 강남 맛집 중 별점 4점 이상, 평균가 20000원 이상, 40000원 이하인 곳 메뉴
r_filter(gangnam_r, gangnam_r_info, star = 4, price_low = 20000, price_high = 40000)

,가게이름,메뉴,가격
2,강남진해장,양선지해장국,"11,000원"
3,강남진해장,곱창전골,"62,000원"
4,강남진해장,양지내장탕,"14,000원"
5,강남진해장,사골곰탕,"11,000원"
6,강남진해장,양지곰탕,"11,000원"
...,...,...,...
485,진미평양냉면,냉면곱빼기,"18,000원"
486,진미평양냉면,만두반,"7,000원"
487,진미평양냉면,어복쟁반 대,"90,000원"
488,진미평양냉면,냉면,"14,000원"
